In [18]:
# Import
import yfinance as yf
import pandas as pd
import numpy as np
import requests
import talib as talib

In [19]:
# import list of top 100 stocks USA
# import stock list from CSV file
importedCsv = pd.read_csv (r'../CSV/StockList.csv', sep=',')
print(importedCsv["Symbol"])

0     AAPL
1     MSFT
2     GOOG
3     AMZN
4     TSLA
      ... 
95      MO
96     ADI
97    GILD
98     TFC
99      GM
Name: Symbol, Length: 100, dtype: object


In [20]:
# Functions
def checkNonesString(df):
    # CHECK Object column for "None" strings.
    print(df[df['Symbol'].str.contains('None')].any())
    

def checkNans(df):
    # Check if there are NaNs
    print("-----IS NA per COL-------")
    print("Doesn't seem to be working")
    print(df[df.isna().any(axis=1)].sum())

    print("-----IS NA per COL-------")
    print(df.isna().sum())

    print("------printing sum of Nans for each column------")
    for col in df.columns:
        print(col + ": " + str(df[col].isnull().sum()))
   
        
def checkInfs(df):
    print("-------total amount of Infs per Column-----------")
    print(df.groupby(np.isinf(df["Open"])).count())

    print("-------total amount of Infs-----------")
    count = np.isinf(df["Open"]).values.sum()
    print("It contains " + str(count) + " infinite values")


def printDataframe(df):
    # Prints
    print("-----DATAFRAME-------")
    print(df.head(10))

    print("-----SHAPE-------")
    print(df.shape)

    print("-----COLUMNS-------")
    print(df.columns)

    print("-----DATATYPES-------")
    print(df.dtypes)



In [21]:
# Add historical data for all stocks in ImportedCSV file = 100 stocks
final_df = pd.DataFrame()

oneStockAAPL = yf.Ticker("AAPL").history(period="max")
# adding symbol for traceability
oneStockAAPL["Symbol"] = "AAPL"
# Rearragning the date column to a functionalt "timestamp"
#oneStockAAPL["Timestamp"] = oneStockAAPL.index
oneStockAAPL.insert(0, 'Timestamp', oneStockAAPL.index)

for col in importedCsv["Symbol"]:
        
    # fetch data for every stock
    temp = yf.Ticker(col).history(start="2015-01-15", end="2022-02-15")
    # adding symbol for traceability
    temp["Symbol"] = col
    # Rearragning the date column to a functionalt "timestamp"
    temp["Timestamp"] = temp.index
    # concateing with the dataframe for all stocks
    final_df = pd.concat([final_df, temp], axis=0)

final_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Symbol,Timestamp
Date,,,,,,,,,
2015-01-14,24.648955,24.976733,24.526886,24.820757,195826400,0.0,0.0,AAPL,2015-01-14
2015-01-15,24.865962,24.879525,24.110942,24.147110,240056000,0.0,0.0,AAPL,2015-01-15
2015-01-16,24.194581,24.318911,23.780901,23.959484,314053200,0.0,0.0,AAPL,2015-01-16
2015-01-20,24.377689,24.633132,24.074777,24.576618,199599600,0.0,0.0,AAPL,2015-01-20
2015-01-21,24.628612,25.105587,24.474895,24.764246,194303600,0.0,0.0,AAPL,2015-01-21


In [22]:
# restting index
oneStockAAPL.reset_index(drop=True, inplace=True)
final_df.reset_index(drop=True, inplace=True)

In [23]:
# Checking functions
printDataframe(final_df)
checkInfs(final_df)
checkNans(final_df)
checkNonesString(final_df)

-----DATAFRAME-------
        Open       High        Low      Close     Volume  Dividends  \
0  24.648955  24.976733  24.526886  24.820757  195826400        0.0   
1  24.865962  24.879525  24.110942  24.147110  240056000        0.0   
2  24.194581  24.318911  23.780901  23.959484  314053200        0.0   
3  24.377689  24.633132  24.074777  24.576618  199599600        0.0   
4  24.628612  25.105587  24.474895  24.764246  194303600        0.0   
5  24.924739  25.424319  24.802670  25.408495  215185600        0.0   
6  25.385891  25.713669  25.211828  25.539608  185859200        0.0   
7  25.711407  25.851561  25.498917  25.566732  222460000        0.0   
8  25.413016  25.426580  24.646692  24.671558  382274800        0.0   
9  26.590757  26.701524  26.066311  26.066311  585908400        0.0   

   Stock Splits Symbol  Timestamp  
0           0.0   AAPL 2015-01-14  
1           0.0   AAPL 2015-01-15  
2           0.0   AAPL 2015-01-16  
3           0.0   AAPL 2015-01-20  
4           0.0 

C:\Users\Eric\AppData\Local\Temp/ipykernel_11560/1751103324.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print(df[df.isna().any(axis=1)].sum())


In [24]:
# Dropping rows with NaNs, (should only be one in NVDA)
final_df.dropna(inplace=True)

# Checking number of tickers in Symbol column
print(final_df["Symbol"].unique())
print(len(final_df["Symbol"].unique()))

['AAPL' 'MSFT' 'GOOG' 'AMZN' 'TSLA' 'FB' 'BRK-A' 'NVDA' 'V' 'JPM' 'JNJ'
 'UNH' 'WMT' 'BAC' 'HD' 'PG' 'MA' 'PFE' 'XOM' 'DIS' 'KO' 'CSCO' 'CVX'
 'ADBE' 'AVGO' 'PEP' 'ABBV' 'CMCSA' 'TMO' 'NKE' 'ORCL' 'LLY' 'NFLX' 'WFC'
 'CRM' 'ABT' 'INTC' 'VZ' 'COST' 'QCOM' 'PYPL' 'DHR' 'MRK' 'T' 'MCD' 'SCHW'
 'UPS' 'MS' 'TXN' 'AMD' 'LOW' 'NEE' 'PM' 'UNP' 'INTU' 'HON' 'AMAT' 'BMY'
 'CVS' 'RTX' 'TMUS' 'C' 'BA' 'AMGN' 'AXP' 'BLK' 'GS' 'CAT' 'IBM' 'SBUX'
 'DE' 'EL' 'COP' 'AMT' 'GE' 'PLD' 'ISRG' 'ANTM' 'MU' 'CHTR' 'TGT' 'NOW'
 'SPGI' 'ABNB' 'MMM' 'LMT' 'LRCX' 'F' 'BKNG' 'SYK' 'ZTS' 'ADP' 'MDLZ'
 'PNC' 'USB' 'MO' 'ADI' 'GILD' 'TFC' 'GM']
100


In [25]:
# Checking functions
printDataframe(final_df)
checkInfs(final_df)
checkNans(final_df)
checkNonesString(final_df)

-----DATAFRAME-------
        Open       High        Low      Close     Volume  Dividends  \
0  24.648955  24.976733  24.526886  24.820757  195826400        0.0   
1  24.865962  24.879525  24.110942  24.147110  240056000        0.0   
2  24.194581  24.318911  23.780901  23.959484  314053200        0.0   
3  24.377689  24.633132  24.074777  24.576618  199599600        0.0   
4  24.628612  25.105587  24.474895  24.764246  194303600        0.0   
5  24.924739  25.424319  24.802670  25.408495  215185600        0.0   
6  25.385891  25.713669  25.211828  25.539608  185859200        0.0   
7  25.711407  25.851561  25.498917  25.566732  222460000        0.0   
8  25.413016  25.426580  24.646692  24.671558  382274800        0.0   
9  26.590757  26.701524  26.066311  26.066311  585908400        0.0   

   Stock Splits Symbol  Timestamp  
0           0.0   AAPL 2015-01-14  
1           0.0   AAPL 2015-01-15  
2           0.0   AAPL 2015-01-16  
3           0.0   AAPL 2015-01-20  
4           0.0 

C:\Users\Eric\AppData\Local\Temp/ipykernel_11560/1751103324.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print(df[df.isna().any(axis=1)].sum())


In [26]:
# Exporting all stocks dataframe to CSV
final_df.to_csv(r'../CSV/all_stocks.csv', sep= ",", index = False)

In [ ]:
# Exporting single stock dataframe to CSV
oneStockAAPL.to_csv(r'../CSV/oneStockAAPL.csv', sep= ",", index = False)